In [1]:
# Import Statements
import eeg_utils
import numpy as np
from eeg_utils import *
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
# Load EEG Data
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [3]:
# Visualize EEG Data Shapes
print("Training/Valid data shape: {}".format(X_train_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Training/Valid target shape: {}".format(y_train_valid.shape))
print("Test target shape: {}".format(y_test.shape))
print("Person train/valid shape: {}".format(person_train_valid.shape))
print("Person test shape: {}".format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [4]:
# Preprocess Data
data_config = {"Duplicate": 2, "Mu": 0.0, "Sigma": 0.5,
               "Sample": False, "Sample_Step": 5, "Attention": False, "Swap": True}
X_train_valid, y_train_valid, X_test, y_test = preprocess_data(X_train_valid,
                                                               y_train_valid,
                                                               X_test,
                                                               y_test, 
                                                               data_config)

In [5]:
# Split Training/Validation Data
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid,
                                                      y_train_valid,
                                                      test_size=0.2,
                                                      random_state=0)

In [6]:
# Trim Data pre-Model
X_train_5d = X_train[:, :500, :, :]
X_valid_5d = X_valid[:, :500, :, :]
X_test_5d = X_test[:, :500, :, :]

In [7]:
# Load Optimal Model
model_name = "./CRNN_Train_All"
optim_model = keras.models.load_model(model_name) # Model that was trained on all subjects

In [8]:
# Evaluate Results (Train on all subjects, Test on all subjects)
optim_model.evaluate(X_train_5d, y_train) # 95.51%
optim_model.evaluate(X_valid_5d, y_valid) # 88.53%
optim_model.evaluate(X_test_5d, y_test) # 70.43%

14/14 [==============================] - 0s 18ms/step - loss: 1.3565 - accuracy: 0.7043


[1.3564612865447998, 0.7042889595031738]

In [9]:
# Separate data by patient
patient_data = []
patient_truths = []

for patient in np.unique(person_test):
    idx = np.where(person_test == patient)[0]
    patient_data.append(X_test_5d[idx, :, :, :])
    patient_truths.append(y_test[idx])
    pass

In [10]:
# Evaluate model on each patient (Train on all subjects, Test on a single subject)
for i in range(len(patient_data)):
    optim_model.evaluate(patient_data[i], patient_truths[i])

2/2 [==============================] - 0s 30ms/step - loss: 1.1758 - accuracy: 0.7234


In [11]:
'''
Model Parameters: 147 Epochs
    - Data augmented CRNN with doubly stacked LSTM layer
    - Regularization : 0.006
    - Learning Rate : 0.001
    - Attention : False
    - Time Points : 500
    - Kernel : (10, 1)
    - Sample : False
    - Dropout : 0.5
    - Pool : (3, 1)
    - Duplicate : 2
    - Sigma : 0.5
    - Mu : 0.0
    
    N.B. For importance of the parameters above, see CRNN Model notebook for architecture
'''

'\nModel Parameters:\n    - Data augmented CRNN with doubly stacked LSTM layer\n    - Regularization : 0.006\n    - Learning Rate : 0.001\n    - Attention : False\n    - Time Points : 500\n    - Kernel : (10, 1)\n    - Sample : False\n    - Dropout : 0.5\n    - Pool : (3, 1)\n    - Duplicate : 2\n    - Sigma : 0.5\n    - Mu : 0.0\n    \n    N.B. For importance of the parameters above, see CRNN Model notebook for architecture\n'